In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Batch Prediction with Model Monitoring

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fbatch_prediction_model_monitoring.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb" target='_blank'>
       <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

In this notebook, you learn how to use Model Monitoring with batch prediction requests on a deployed Vertex AI model resource. In a companion notebook, <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb" target="_blank">Vertex AI Model Monitoring with Explainable AI feature attributions</a>, you can learn about how to apply model monitoring to streaming, real-time predictions.

Learn more about [Vertex AI Model Monitoring for batch predictions](https://cloud.google.com/vertex-ai/docs/model-monitoring/model-monitoring-batch-predictions).

### Objective
In this notebook, you learn to use the Vertex AI model monitoring service to detect drift and anomalies in batch prediction.

This tutorial uses the following Google Cloud ML services:

- Vertex AI model monitoring
- Vertex AI batch prediction
- Vertex AI model resource

The steps performed include:

- Upload a pre-trained model as a Vertex AI model resource.
- Generate batch prediction requests.
- Interpret the statistics, visualizations, other data reported by the model monitoring feature.

### Model

This tutorial uses a pre-trained model, where the model artifacts are stored in a public Cloud Storage bucket. The model predicts for an online gaming site, the probability that a player may churn, i.e. stop being an active player.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertext AI
* Google Cloud Storage

Learn about [Vertext AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### What is Vertex AI batch prediction?

<a href="https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions" target="_blank">Batch Prediction</a> is a service that processes a collection (i.e. a batch) of machine learning inference requests in bulk, with less stringent response time requirements than real-time or streaming predictions.

### What is Vertex AI model monitoring?

<a href="https://cloud.google.com/vertex-ai/docs/model-monitoring" target="_blank">Model Monitoring</a> is a service that automatically determines 
whether production machine learning traffic differs from  training data, or varies substantially over time, in terms of model predictions or feature attributions. When that happens, you can be alerted automatically and responsively, whereby, you can detect model decay which may negatively impact your operations -- such as your customer experience and/or revenue.

### The example model

The model you use in this notebook is based on [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml). The idea behind this model is that your company has extensive log data describing how your game users have interacted with the site. The raw data contains the following categories of information:

- identity - unique player identitity numbers
- demographic features - information about the player, such as the geographic region in which a player is located
- behavioral features - counts of the number of times a  player has triggered certain game events, such as reaching a new level
- churn propensity - this is the label or target feature, it provides an estimated probability that this player churns, i.e. stop being an active player.

The blog article referenced above explains how to use BigQuery to store the raw data, pre-process it for use in machine learning, and train a model. Because this notebook focuses on model monitoring, rather than training models, you're going to reuse a pre-trained version of this model, which has been exported to Google Cloud Storage. In the next section, you setup your environment and import this model into your own project.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
# Install Python package dependencies.
! pip3 install --upgrade --quiet google-cloud-aiplatform

! pip3 install --upgrade --quiet tensorflow-data-validation

! pip3 install --quiet cachetools==5.2.0

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type:"string"}

#### Set your email address
This is used for delivering model monitoring notifications.


In [ ]:
import os

EMAIL_ADDRESS = "[your-email-address]"  # @param {type:"string"}
if not EMAIL_ADDRESS or EMAIL_ADDRESS == "[your-email-address]":
    if os.getenv("IS_TESTING"):
        EMAIL_ADDRESS = "noreply@google.com"
    else:
        print("EMAIL_ADDRESS not specified, please correct before proceeding.")

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
import time

import google.cloud.aiplatform as aiplatform
import tensorflow_data_validation as tfdv
from google.cloud.aiplatform_v1beta1.services.job_service import \
    JobServiceClient
from google.cloud.aiplatform_v1beta1.types import (
    BatchDedicatedResources, BatchPredictionJob, GcsDestination, GcsSource,
    MachineSpec, ModelMonitoringAlertConfig, ModelMonitoringConfig,
    ModelMonitoringObjectiveConfig, ThresholdConfig)
from tensorflow_data_validation.utils import io_util
from tensorflow_metadata.proto.v0 import statistics_pb2

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for prediction (e.g., GPUs) or choose not to use any (CPU). Hardware accelertors lower the latency response for a prediction request. When choosing a hardware accelerators, consider the additional cost trade-off over latency.

Set the variables `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Tesla T4 GPUs allocated to each VM, you would specify:

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

See the [locations where accelerators are available](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

Otherwise specify `(None, None)` to use a container image to run on a CPU.

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.

For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-13"
else:
    DEPLOY_VERSION = "tf2-cpu.2-13"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### Set machine types

Next, set the machine types to use for training and prediction.

- Set the variable `DEPLOY_COMPUTE` to configure your compute resources for prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

### Upload the model artifacts as a Vertex AI model resource

First, you upload the pre-trained custom tabular model artifacts as a Vertex AI model resource using the `upload()` method, with the following parameters:

- `display_name`: The human readable name for the model resource.
- `artifact_uri`: The Cloud Storage location of the model artifacts.
- `serving_container_image`: The serving container image to use when the model is deployed to a Vertex AI endpoint resource.
- `sync`: Whether to wait for the process to complete, or return immediately (async).

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

model = aiplatform.Model.upload(
    display_name="churn",
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

## Submit a batch prediction request with model monitoring enabled


### Create Cloud Storage buckets

This cell creates two Cloud Storage buckets:

- `gs://PROJECT_ID_bp_mm_input` contains the batch prediction request data.
- `gs://PROJECT_ID_bp_mm_output` contains the batch prediction output as well as the model monitoring results.

In [ ]:
# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
OUTPUT_GS_PATH = f"{BUCKET_URI}/bp_mm_output"
INPUT_GS_PATH = f"{BUCKET_URI}/bp_mm_input"
PUBLIC_TRAINING_DATASET = "gs://bp_mm_public_data/churn/churn_bp_insample.csv"
TRAINING_DATASET = f"{INPUT_GS_PATH}/churn_bp_insample.csv"
TRAINING_DATASET_FORMAT = "csv"

! gsutil copy $PUBLIC_TRAINING_DATASET $TRAINING_DATASET

### Create batch prediction job

This step parameterizes and builds the data structure representing the batch prediction request, with model monitoring enabled.

The BatchPredictionJob object specifies the input source, the data format, and the computing resources requests for the batch prediction. Learn more about <a href="https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-sample" target="_blank">BatchPredictionJob</a>.

The ModelMonitoringConfig object specifies the alerting email address, the training dataset, the features to be monitored, and their associated alerting thresholds. Learn more about <a href="https://cloud.google.com/vertex-ai/docs/model-monitoring" target="_blank">ModelMonitoringConfig</a>.

In [ ]:
INPUT_URI = "gs://bp_mm_public_data/churn/churn_bp_outsample.jsonl"
OUTPUT_URI = OUTPUT_GS_PATH
INSTANCES_FORMAT = "jsonl"
PREDICTIONS_FORMAT = "jsonl"
JOB_NAME_PREFIX = "bp_mm_demo"
MODEL_NAME = model.resource_name
BATCH_PREDICTION_JOB_NAME = JOB_NAME_PREFIX

batch_prediction_job = BatchPredictionJob(
    display_name=BATCH_PREDICTION_JOB_NAME,
    model=MODEL_NAME,
    input_config=BatchPredictionJob.InputConfig(
        instances_format=INSTANCES_FORMAT, gcs_source=GcsSource(uris=[INPUT_URI])
    ),
    output_config=BatchPredictionJob.OutputConfig(
        predictions_format=PREDICTIONS_FORMAT,
        gcs_destination=GcsDestination(output_uri_prefix=OUTPUT_URI),
    ),
    dedicated_resources=BatchDedicatedResources(
        machine_spec=MachineSpec(machine_type=DEPLOY_COMPUTE),
        starting_replica_count=1,
        max_replica_count=1,
    ),
    # Model monitoring service is triggerred if provide following configs.
    model_monitoring_config=ModelMonitoringConfig(
        alert_config=ModelMonitoringAlertConfig(
            email_alert_config=ModelMonitoringAlertConfig.EmailAlertConfig(
                user_emails=[EMAIL_ADDRESS]
            )
        ),
        objective_configs=[
            ModelMonitoringObjectiveConfig(
                training_dataset=ModelMonitoringObjectiveConfig.TrainingDataset(
                    data_format=TRAINING_DATASET_FORMAT,
                    gcs_source=GcsSource(uris=[TRAINING_DATASET]),
                ),
                training_prediction_skew_detection_config=ModelMonitoringObjectiveConfig.TrainingPredictionSkewDetectionConfig(
                    skew_thresholds={
                        "cnt_user_engagement": ThresholdConfig(value=0.001),
                        "julianday": ThresholdConfig(value=0.001),
                    }
                ),
            )
        ],
    ),
)

### Submit batch prediction job

This step submits the batch prediction request created in the previous step. If successful, it returns a JSON document summarizing the request, which is displayed in the cell output below.

In [ ]:
API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
client = JobServiceClient(client_options={"api_endpoint": API_ENDPOINT})
out = client.create_batch_prediction_job(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}",
    batch_prediction_job=batch_prediction_job,
)
BATCH_PREDICTION_JOB_ID = out.name.split("/")[-1]
print("BATCH_PREDICTION_JOB_ID:", BATCH_PREDICTION_JOB_ID)

## Verify prediction results

The batch prediction request gets completed after about **17 mins**, and the model monitoring result is available **10 mins** post that. The request below obtains the batch prediction job id, which is a unique number associated with asynchronous requests like this one.

In [ ]:
# If auto-testing, wait for request completion
if os.getenv("IS_TESTING"):
    time.sleep(3000)

### Browse storage bucket

Click on the link below, which opens the Cloud Storage object viewer on the output bucket you created above, to see the results of your batch prediction request.

When everything is ready, you see two folders in the bucket:

- `prediction-batch_prediction_monitoring_test_model_<timestmap>` - this folder contains the your batch prediction results, i.e. the predictions produced by your model for each input in the batch
- `job-<id>` - this folder contains the model monitoring results, including the model schema, monitoring thresholds and other config settings, statistics, and anomalies

*Note:* Until the request and the monitoring job are completed, you may see empty or partial results in this bucket.

In [ ]:
print(f"https://console.cloud.google.com/storage/browser/{OUTPUT_URI.lstrip('gs://')}")

### Visualize the batch prediction result

Run the following cell to examine the batch prediction results with a tabular and visual analysis using the <a href="https://www.tensorflow.org/tfx/data_validation/get_started" target="_blank">TensorFlow Data Validation</a> package

In [ ]:
! rm -f ./training_stats.pb
! rm -f ./prediction_stats.pb

TRAINING_STATS_SUBPATH = "stats_training/stats/training_stats"
PREDICTION_STATS_SUBPATH = "stats_and_anomalies/stats/current_stats"
STATS_GCS_FOLDER = OUTPUT_URI = (
    OUTPUT_GS_PATH + "/job-" + BATCH_PREDICTION_JOB_ID + "/bp_monitoring/"
)
TRAINING_STATS_GCS_PATH = STATS_GCS_FOLDER + TRAINING_STATS_SUBPATH
print("Looking up statistics from: " + TRAINING_STATS_GCS_PATH)
PREDICTION_STATS_GCS_PATH = STATS_GCS_FOLDER + PREDICTION_STATS_SUBPATH
print("Looking up statistics from: " + PREDICTION_STATS_GCS_PATH)

! gsutil cp $TRAINING_STATS_GCS_PATH ./training_stats.pb
! gsutil cp $PREDICTION_STATS_GCS_PATH ./prediction_stats.pb


# util function to load stats binary file from GCS
def load_stats_binary(input_path):
    stats_proto = statistics_pb2.DatasetFeatureStatisticsList()
    stats_proto.ParseFromString(
        io_util.read_file_to_string(input_path, binary_mode=True)
    )
    return stats_proto


tfdv.visualize_statistics(load_stats_binary("./training_stats.pb"))
tfdv.visualize_statistics(load_stats_binary("./prediction_stats.pb"))

### Check skew results

Finally, you can check the skew results by examining a file in the output bucket. The JSON file contains a report indicating how much the batch prediction data deviates from the training data, on a feature-by-feature basis.

In [ ]:
SKEW_GS_PATH = (
    STATS_GCS_FOLDER
    + "stats_and_anomalies/anomalies/training_prediction_skew_anomalies"
)
! gsutil cat $SKEW_GS_PATH

## Learn more

**Congratulations!** You've now learned how to monitor batch predictions, and how to find and interpret the results. Check out the following resources to learn more about model monitoring and ML Ops.

- [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv)
- [Data Understanding, Validation, and Monitoring At Scale](https://blog.tensorflow.org/2018/09/introducing-tensorflow-data-validation.html)
- [Vertex Product Documentation](https://cloud.google.com/vertex-ai)
- [Vertex AI Model Monitoring Reference docs](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.modelDeploymentMonitoringJobs)
- [Vertex AI Model Monitoring blog article](https://cloud.google.com/blog/topics/developers-practitioners/monitor-models-training-serving-skew-vertex-ai)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Vertex AI Model
- Batch prediction job
- Cloud Storage bucket (set `delete_bucket` to *True* for deletion)

In [ ]:
# Delete model resource
model.delete()

# Get the batch prediction job resource
batch_job = aiplatform.BatchPredictionJob(
    batch_prediction_job_name=BATCH_PREDICTION_JOB_ID
)
# Delete the job
batch_job.delete()

# Delete Cloud Storage bucket
delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

! rm -f ./training_stats.pb
! rm -f ./prediction_stats.pb